In [0]:
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from scipy.optimize import differential_evolution

import os
import re
import csv
import codecs
import numpy as np
import pandas as pd
import operator
import sys

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from __future__ import absolute_import, division

import tensorflow as tf
from keras.layers import Dense, Input, Embedding, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

Using TensorFlow backend.


In [0]:
###################Two level authentication for mounting the drive content on your 
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print('Files in Drive:')
!ls drive/

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130791 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force

Enter verification code: ··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3A

In [0]:
#Change the file paths depending upon your own drives path
path = 'Files/'
EMBEDDING_FILE='drive/finalYrProject/Ensemble/Copy of crawl-300d-2M.vec'    #Embedding/crawl-300d-2M.vec'
TRAIN_DATA_FILE='drive/finalYrProject/Toxic_CC_2/Dataset/cleaned_train.csv'    #path + 'cleaned_train.csv'
TEST_DATA_FILE='drive/finalYrProject/Toxic_CC_2/Dataset/cleaned_test.csv'            #path + 'test.csv'

MAX_SEQUENCE_LENGTH = 400 #max length of sequence
MAX_NB_WORDS = 100000 #Top count of words under consideration
EMBEDDING_DIM = 300 #the dimension of the word embeddings

drive/finalYrProject/Ensemble/Copy of crawl-300d-2M.vec
/content


In [0]:
from drive.finalYrProject.Toxic_CC_1.toxic_m.data_helper.data_loader import DataLoader
data_loader = DataLoader()
embeddings_index = data_loader.load_embedding(EMBEDDING_FILE)
print(type(embeddings_index))

Total 2000000 word vectors.
<class 'dict'>


In [0]:
train_df = pd.read_csv(TRAIN_DATA_FILE)
test_df = pd.read_csv(TEST_DATA_FILE)



In [0]:
import drive.finalYrProject.Toxic_CC_1.toxic_m.models.keras.model_set as model_set

In [0]:
########################################
## process texts in datasets
########################################
print('Processing text dataset')
#replace the null coment to no comments
list_sentences_train = train_df["comment_text"].fillna("no comment").values
#list of final classification classes
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y = train_df[list_classes].values


list_sentences_test = test_df["comment_text"].fillna("no comment").values
#create a list of comments from training data set 
comments = []
for text in list_sentences_train:
    comments.append(text)
#create a list of comments from testing data set
test_comments=[]
for text in list_sentences_test:
    test_comments.append(text)
###########################################################
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
#Fitting the text on
tokenizer.fit_on_texts(comments + test_comments)
###########################################################
sequences = tokenizer.texts_to_sequences(comments)
test_sequences = tokenizer.texts_to_sequences(test_comments)
###########################################################
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))
###########################################################
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

Processing text dataset
Found 332116 unique tokens
Shape of data tensor: (159571, 400)
Shape of label tensor: (159571, 6)


In [0]:
#############################validation_data contains the data against which we cross verify the result
validation_data = data
print('Shape of validation_data tensor:', validation_data.shape)

Shape of validation_data tensor: (159571, 400)


In [0]:
#############################validation_data contains the data against which we cross verify the result
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of test_data tensor:', test_data.shape)

Shape of test_data tensor: (153164, 400)


In [0]:
########################################
## prepare embeddings
########################################
print('Preparing embedding matrix')
nb_words = min(MAX_NB_WORDS, len(word_index))
#embedding_matrix = np.random.normal(loc=matrix_mean, scale=matrix_std, size=(nb_words, EMBEDDING_DIM))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
null_count = 0
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        null_count += 1
print('Null word embeddings: %d' % null_count)

Preparing embedding matrix
Null word embeddings: 24105


In [0]:
#using same data for validation and training to get the local AUC ROC score
validation_y=y

In [0]:
models=[]
#kmax_text
model_kmax_text_1=model_set.get_kmax_text_cnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_kmax_text_2=model_set.get_kmax_text_cnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_kmax_text_3=model_set.get_kmax_text_cnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_kmax_text_4=model_set.get_kmax_text_cnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_kmax_text_5=model_set.get_kmax_text_cnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
#av_rnn
model_av_rnn_1=model_set.get_av_rnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_av_rnn_2=model_set.get_av_rnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_av_rnn_3=model_set.get_av_rnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_av_rnn_4=model_set.get_av_rnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_av_rnn_5=model_set.get_av_rnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
#dropout_bi_gru
model_dropout_gru_1=model_set.get_dropout_bi_gru(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_dropout_gru_2=model_set.get_dropout_bi_gru(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_dropout_gru_3=model_set.get_dropout_bi_gru(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_dropout_gru_4=model_set.get_dropout_bi_gru(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_dropout_gru_5=model_set.get_dropout_bi_gru(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
#rcnn
model_rnn_1=model_set.get_rcnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_rnn_2=model_set.get_rcnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_rnn_3=model_set.get_rcnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_rnn_4=model_set.get_rcnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)
model_rnn_5=model_set.get_rcnn(nb_words, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, out_size=6)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 400, 300)     30000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 400, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)    

In [0]:
#ploting the model
from keras.utils import plot_model
plot_model(model_kmax_text_1, to_file='drive/finalYrProject/Ensemble/kmax_text_cnn.png')
plot_model(model_av_rnn_1, to_file='drive/finalYrProject/Ensemble/av_rnn.png')
plot_model(model_dropout_gru_1, to_file='drive/finalYrProject/Ensemble/dropout_bi_gru.png')
plot_model(model_rnn_1, to_file='drive/finalYrProject/Ensemble/model_rcnn.png')

In [0]:
#kmax_text
model_kmax_text_1.load_weights('drive/finalYrProject/Toxic_CC_1/kmax_text_cnn_new0.h5')
model_kmax_text_2.load_weights('drive/finalYrProject/Toxic_CC_1/kmax_text_cnn_new1.h5')
model_kmax_text_3.load_weights('drive/finalYrProject/Toxic_CC_1/kmax_text_cnn_new2.h5')
model_kmax_text_4.load_weights('drive/finalYrProject/Toxic_CC_1/kmax_text_cnn_new3.h5')
model_kmax_text_5.load_weights('drive/finalYrProject/Toxic_CC_1/kmax_text_cnn_new4.h5')
#av_rnn
model_av_rnn_1.load_weights('drive/finalYrProject/Toxic_CC_1/model_rnn_check_new0.h5')
model_av_rnn_2.load_weights('drive/finalYrProject/Toxic_CC_1/model_rnn_check_new1.h5')
model_av_rnn_3.load_weights('drive/finalYrProject/Toxic_CC_1/model_rnn_check_new2.h5')
model_av_rnn_4.load_weights('drive/finalYrProject/Toxic_CC_1/model_rnn_check_new3.h5')
model_av_rnn_5.load_weights('drive/finalYrProject/Toxic_CC_1/model_rnn_check_new4.h5')
#dropout_bi_gru
model_dropout_gru_1.load_weights('drive/finalYrProject/Toxic_CC_2/dropout_bi_gru0.h5')
model_dropout_gru_2.load_weights('drive/finalYrProject/Toxic_CC_2/dropout_bi_gru1.h5')
model_dropout_gru_3.load_weights('drive/finalYrProject/Toxic_CC_2/dropout_bi_gru2.h5')
model_dropout_gru_4.load_weights('drive/finalYrProject/Toxic_CC_2/dropout_bi_gru3.h5')
model_dropout_gru_5.load_weights('drive/finalYrProject/Toxic_CC_2/dropout_bi_gru4.h5')
#rnn
model_rnn_1.load_weights('drive/finalYrProject/TCC_3/rcnn0.h5')
model_rnn_2.load_weights('drive/finalYrProject/TCC_3/rcnn1.h5')
model_rnn_3.load_weights('drive/finalYrProject/TCC_3/rcnn2.h5')
model_rnn_4.load_weights('drive/finalYrProject/TCC_3/rcnn3.h5')
model_rnn_5.load_weights('drive/finalYrProject/TCC_3/rcnn4.h5')


In [0]:
###############add the kmax model to a models list#####################
models.append(model_kmax_text_1)
models.append(model_kmax_text_2)
models.append(model_kmax_text_3)
models.append(model_kmax_text_4)
models.append(model_kmax_text_5)
###############add the kmax model to a models list#####################
models.append(model_av_rnn_1)
models.append(model_av_rnn_2)
models.append(model_av_rnn_3)
models.append(model_av_rnn_4)
models.append(model_av_rnn_5)
###############add the kmax model to a models list#####################
models.append(model_dropout_gru_1)
models.append(model_dropout_gru_2)
models.append(model_dropout_gru_3)
models.append(model_dropout_gru_4)
models.append(model_dropout_gru_5)
###############add the kmax model to a models list#####################
models.append(model_rnn_1)
models.append(model_rnn_2)
models.append(model_rnn_3)
models.append(model_rnn_4)
models.append(model_rnn_5)

In [0]:
# make an ensemble prediction for multi-class classification
def ensemble_predictions(models, weights, test_data):
    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict(test_data, batch_size=256, verbose=1)
        test_predicts_list.append(test_predicts)
        np.save("drive/finalYrProject/Ensemble", test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    i=0
    for fold_predict in test_predicts_list:
        test_predicts += weights[i]*fold_predict
        i=i+1
        #here we will have to make the necessary changes 
#    test_predicts /= len(test_predicts_list)
    test_predicts /= sum(weights) 
    print(len(test_predicts))
    return test_predicts


In [0]:
# # evaluate a specific number of members in an ensemble
def evaluate_ensemble(models, weights, val_X, val_y):
  yhat = ensemble_predictions(models, weights, val_X)
  print(val_y)
  print(yhat)
  auc = roc_auc_score(val_y, yhat)
  print("AUC Score", auc)
  return auc

In [0]:
def normalize(weights):
	# calculate l1 vector norm
	result = norm(weights, 1)
	# check for a vector of all zeros
	if result == 0.0:
		return weights
	# return normalized vector (unit norm)
	return weights / result

In [0]:
# loss function for optimization process, designed to be minimized
def loss_function(weights, models, val_x, val_y):
	# normalize weights
	normalized = normalize(weights)
	# calculate error rate
	return 1.0 - evaluate_ensemble(models, normalized, val_x, val_y)

In [0]:
from sklearn.metrics import roc_auc_score

In [0]:
weights = [1.0/len(models) for _ in range(len(models))]
score = evaluate_ensemble(models, weights, validation_data, validation_y)
print('Equal Weights Score: %.3f' % score)

Predicting testing results...
159571/159571 [==============================] - 27s 167us/step
159571
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[1.64222275e-03 2.62558461e-06 1.50029364e-04 6.02751984e-06
  8.25881981e-05 1.79007652e-05]
 [2.76750332e-04 1.59442426e-06 6.89506537e-05 1.83284286e-06
  3.71903188e-05 5.89936981e-06]
 [3.47392265e-03 3.08156019e-06 1.50889160e-04 8.70525863e-06
  1.24301019e-04 2.04592949e-05]
 ...
 [1.61921712e-02 2.14189292e-05 2.01580379e-03 1.51261690e-05
  9.37555753e-04 1.39202183e-04]
 [2.06580164e-03 7.99894358e-06 2.05025079e-04 5.15043746e-05
  2.13094058e-04 3.09869654e-05]
 [1.21613296e-02 9.29832476e-06 3.89079760e-04 9.01430865e-05
  7.65054578e-04 6.72922856e-05]]
AUC Score 0.9939447783336686
Equal Weights Score: 0.994


In [0]:
###############################Run only in case u have a System with GPU which can be kept running for durations more than 30 hours.... optimization portion .......Part of the futute work of the submitted project
####################################Depreciated Codes#############################################
weights = [1.0/len(models) for _ in range(len(models))]
score = evaluate_ensemble(models, weights, validation_data, validation_y)
print('Equal Weights Score: %.3f' % score)

# define bounds on each weight
bound_w = [(0.0, 1.0)  for _ in range(len(models))]

# arguments to the loss function
search_arg = (models, validation_data, validation_y)

# global optimization of ensemble weights
result = differential_evolution(loss_function, bound_w, search_arg, maxiter=5, tol=1e-3,popsize=5)

# get the chosen weights
weights = normalize(result['x'])
print('Optimized Weights: %s' % weights)

# evaluate chosen weights
score = evaluate_ensemble(members, weights, validation_data, validation_y)
print('Optimized Weights Score: %.3f' % score)

Predicting testing results...
159571/159571 [==============================] - 42s 263us/step
159571
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[5.91838872e-03 1.43051154e-06 3.61417739e-04 3.45955298e-05
  2.08929185e-04 2.68618268e-05]
 [1.63524356e-03 5.16573611e-07 4.13606575e-05 2.58783507e-06
  2.37822545e-05 4.58459067e-06]
 [1.16970642e-03 2.73187961e-06 4.06305005e-05 2.64460845e-04
  2.30173279e-05 3.71237605e-05]
 ...
 [5.29408924e-01 2.27103147e-03 1.21795738e-02 5.19299963e-02
  7.31565533e-02 5.44463616e-03]
 [3.77318230e-01 1.39864786e-03 2.49173797e-02 2.68495090e-03
  1.06734043e-01 1.83453953e-01]
 [4.82141092e-01 2.09738818e-04 9.66004422e-02 3.30964730e-04
  4.50864974e-01 7.06394825e-04]]
AUC Score 0.5319347662902775
Equal Weights Score: 0.532
Predicting testing results...
159571/159571 [==============================] - 42s 261us/step
159571
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 0

In [0]:
#test_data = test_df
CLASSES = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
submit_path_prefix = "Ensemble_model"

print("Predicting testing results...")
test_predicts_list = []
for fold_id, model in enumerate(models):
    test_predicts = model.predict(test_data, batch_size=256, verbose=1)
    test_predicts_list.append(test_predicts)
    np.save("pred test_predicts)

test_predicts = np.zeros(test_predicts_list[0].shape)
for fold_predict in test_predicts_list:
    test_predicts += fold_predict
test_predicts /= len(test_predicts_list)

test_ids = test_df["id"].values
test_ids = test_ids.reshape((len(test_ids), 1))

test_predicts = pd.DataFrame(data=test_predicts, columns=CLASSES)
test_predicts["id"] = test_ids
test_predicts = test_predicts[["id"] + CLASSES]
submit_path = submit_path_prefix + "-L{:4f}-A{:4f}.csv".format(val_loss, total_auc)
test_predicts.to_csv(submit_path, index=False)